In [ ]:
# Import libraries
from GRANTA_MIScriptingToolkit import granta as mpy
import GRANTA_MIScriptingToolkit as gdl
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import pandas as pd

In [ ]:
mi = mpy.connect('http://azewacadmi1v1.win.ansys.com/mi_servicelayer/', autologon=True)
mi
mi.dbs

In [ ]:
my_db = mi.get_db('Academic Resources Database')
my_db.tables

In [ ]:
my_db.tables[0].children

In [ ]:
session = gdl.GRANTA_MISession('http://azewacadmi1v1.win.ansys.com/mi_servicelayer/', autoLogon=True)
browseService = session.browseService
databases = browseService.GetDatabases().databases

print("Found {0} databases on the Granta MI Server".format(len(databases)))
for d in databases:
    print("Database key: {0.DBKey}, Database name: {0.name}".format(d))

In [ ]:
df = pd.DataFrame({'DBKey': [db.DBKey for db in databases],
                  'DBName': [db.name for db in databases]})
df

In [ ]:
# Get Tables from Ref Papers DB
dbKey = "MI_RefPapers"
tables = browseService.GetTables(gdl.GetTables(DBKey=dbKey)).tableDetails

print("Found {0} tables in database {1}".format(len(tables), dbKey))
for t in tables:
    print("Table name: {0}".format(t.tableReference.name))

In [ ]:
# Get data from Granta ref papers table
table = 'Granta Reference Papers'
attribute = "Year of Publication"

tableRef = gdl.PartialTableReference(tableName=table)
attrRef = gdl.AttributeReference(name=attribute, DBKey=dbKey, partialTableReference=tableRef)
searchCriterion = gdl.RecordSearchCriterion(searchAttribute=attrRef, existsSearchValue=gdl.ExistsSearchValue())
request = gdl.CriteriaSearch(DBKey=dbKey, searchCriteria=[searchCriterion])

searchResults = session.searchService.CriteriaSearch(request).searchResults

df2 = pd.DataFrame({'ShortName': [r.shortName for r in searchResults],
                     'LongName': [r.longName  for r in searchResults]})
df2

In [ ]:
# Year analysis
attribute = "Year of Publication"
attributeLim = "Principal Granta Product used"

attributesInt = ["Year of Publication"]
attrRefs = [gdl.AttributeReference(name=a, DBKey=dbKey, partialTableReference=tableRef) for a in attributesInt]
recordRefs = [r.recordReference for r in searchResults]
request = gdl.GetRecordAttributesByRefRequest(recordReferences=recordRefs, attributeReferences=attrRefs)
recordData = session.dataExportService.GetRecordAttributesByRef(request).recordData

# Export data in a table
s_year = [None]*len(df2)
for attributeInt in attributesInt:
    for idx, record in enumerate(recordData):
        attrValueInt = next((x for x in record.attributeValues if x.attributeName == attributeInt), None)
        s_year[idx] = attrValueInt.integerDataValue.value.real if attrValueInt else None
    df2[attributeInt] = s_year
df2

#Plot
figure=plt.figure(1)
plt.hist(s_year, bins=max(s_year)-min(s_year))
plt.xlabel('Year of Publication')
plt.ylabel('Number of Paper')
plt.savefig("Paper_Year"+".png", bbox_inches='tight')